# NRPy+ Tutorial

**[Zachariah B. Etienne](http://math.wvu.edu/~zetienne/)** $\leftarrow$ Please feel free to email comments, revisions, or errata!

In [1]:
from NRPy_logo import *
print_logo()

ooooo      ooo ooooooooo.   ooooooooo.                 88
`888b.     `8' `888   `Y88. `888   `Y88.             888888  
 8 `88b.    8   888   .d88'  888   .d88' oooo    ooo   88
 8   `88b.  8   888ooo88P'   888ooo88P'   `88.  .8'
 8     `88b.8   888`88b.     888           `88..8'
 8       `888   888  `88b.   888            `888'
o8o        `8  o888o  o888o o888o            .8'
                                         .o..P'
  NRPy+: Python-based Code Generation    `Y8P'
   for Numerical Relativity... and Beyond!



## Introduction & Motivation

[NRPy+](http://blackholesathome.net) ("Python-based code generation for numerical relativity... and beyond!") is a Python-based code generation package: users input complex tensorial expressions in clean tensorial form, and NRPy+ generates highly-optimized C code.

Though it was originally designed to greatly ease the burden on developers when writing codes to numerically solve Einstein's equations of general relativity on the computer, NRPy+ can be used more broadly to rapidly develop codes to solve any number of [hyperbolic](https://en.wikipedia.org/wiki/Hyperbolic_partial_differential_equation) and [parabolic](https://en.wikipedia.org/wiki/Parabolic_partial_differential_equation) partial differential equations with finite difference approaches.

## Interactive NRPy+ Tutorials
#### (READ FIRST, IN ORDER) Basic functionality of NRPy+
+ #### (90% complete) [Basic C Code Output Options (including SIMD), NRPy+'s Parameter Interface](Tutorial-Coutput__Parameter_Interface.ipynb)
+ #### (95% complete) [Indexed Expressions (e.g., tensors, pseudotensors, etc.)](Tutorial-Indexed_Expressions.ipynb)
+ #### (0% complete) [Numerical Grids](Tutorial-Numerical_Grids.ipynb)
+ #### (70% complete) [Finite Difference Derivatives](Tutorial-Finite_Difference_Derivatives.ipynb)
+ #### (0% complete) [Moving beyond Cartesian Grids: Reference Metric](Tutorial-Reference_Metric.ipynb)

#### (READ SECOND, IN ORDER) Worked Examples
+ #### (80% complete) [The Scalar Wave Equation (Cartesian Coordinates)](Tutorial-ScalarWave.ipynb)
+ #### (0% complete) [The Scalar Wave Equation in Curvilinear Coordinates](Tutorial-ScalarWave-Curvi.ipynb)
+ #### (0% complete) [General Relativity in the BSSN Formalism, Moving Puncture Gauge](Tutorial-BSSN.ipynb)